<h1 class='alert alert-success'>Alex Hernandez - INFO THEORY HW #3</h1>

##4.3 with Huffman Code

In [84]:
## Assume both probability distrubutions are a dictionary
function cartPD(p::Dict , k::Dict; rnd = 5 )
    pd = Dict()
    for (psym, pprob) in p, (ksym, kprob) in k
        push!( pd, psym*ksym, round(pprob*kprob,rnd) )
    end
    
    return pd
end

cartPD (generic function with 1 method)

In [85]:
pd_34 = {"A"=>0.8,"B"=>0.2}

{"B"=>0.2,"A"=>0.8}

In [86]:
pd2_34 = cartPD(pd_34, pd_34)

{"AA"=>0.64,"BB"=>0.04,"AB"=>0.16,"BA"=>0.16}

In [87]:
pd3_34 = cartPD(pd2_34, pd_34)

{"BBB"=>0.008,"BAA"=>0.128,"AAB"=>0.128,"AAA"=>0.512,"BBA"=>0.032,"ABB"=>0.032,"BAB"=>0.032,"ABA"=>0.128}

In [167]:
function entropy(pd::Dict{Any,Any}, base=2)
    h = 0.0
    r = log2(length(pd))
    
    for (s_i,p_i) in pd
        h += (p_i * log(base,(1.0/p_i)))
    end
    
    return h/r
end
    

entropy (generic function with 2 methods)

In [168]:
entropy(pd_34)

0.7219280948873623

In [169]:
entropy(pd2_34)

0.7219280948873624

In [170]:
entropy(pd3_34)

0.7219280948873624

In [183]:
# returns the least value of dictionary enteries
function minvalue(pd)
    runonce = false
    
    (kmin,cmin) = (nothing,nothing)
    
    for (k,v) in pd
        if( !runonce )
            runonce = true
            (kmin,cmin) = (k,v)
        elseif( v <= cmin)
            (kmin, cmin) = (k,v)
        end
    end
    
    return (kmin,cmin)
end

minvalue (generic function with 1 method)

In [182]:
# IGNORE THIS !!!!
# This was a first attempt, in coding the second __while__ statement I realized that
# it could substitute for the ALL the code. In fact, the huffmanCode() algorithm is
# just a copy/paste of the second __while__ loop.
function HuffmanTreeFirstAttempt( pd ; rnd=5 )
    init_pd = copy(pd)
    huffman_tree = Dict()
    
    while(length(init_pd) > 0)
        
        #find the smallest probability in the probability distrabution
        cmb_prob = 0.0
        pair = Dict()
        
        #get first new item
        m = (k,v) = minvalue(init_pd) # get the smallest value form the initial PD
        push!(pair,k,v) # push that value
        cmb_prob += pop!(init_pd,k)
        
        # if the init_PD is empty just add minvalue from Huffman
        if( length(init_pd) == 0)
            m = (k,v) = minvalue(huffman_tree)
            push!(pair,k,v)
            cmb_prob += pop!(huffman_tree,k)
            push!(huffman_tree,pair,cmb_prob)
            
        end
        
        #do it a second time
        m = (k,v) = minvalue(init_pd)
        push!(pair,k,v)
        cmb_prob += pop!(init_pd,k)
        
        # loop through the pair of values to find if there exists a value in the 
        # huffman tree which is smaller ( it is impossible that both values in the
        # the __pair__ are less than value in the huffman tree )
        for (k,v) in pair, (s,t) in huffman_tree
            if( t < v)
                push!(init_pd,k,v) # return the value back to the PD
                cmb_prob -= pop!(pair,k) # remove the lesser prob, removing its contrib to cmb_prob
                push!(pair,s,t)
                cmb_prob += pop!(huffman_tree,s) 
                break
            end
        end
        
        # In the first case, when huffman is empty, the for loop will just skip
        # and thats fine because we've already calculated what we need
        
        push!(huffman_tree,pair,round(cmb_prob,rnd))
    end
    
    # now the case happens that all pairs have been added to the huffman tree
    # but that does mean everything is done, the huffman tree can remain
    while(length(huffman_tree) > 1)
        cmb_prob = 0.0
        pair = Dict()
        
        #get first new item
        m = (k,v) = minvalue(huffman_tree) # get the smallest value form the huffman tree
        push!(pair,k,v) 
        cmb_prob += pop!(huffman_tree,k)
        
        m = (k,v) = minvalue(huffman_tree) # get the smallest value form the huffman tree
        push!(pair,k,v) 
        cmb_prob += pop!(huffman_tree,k)
        
        push!(huffman_tree,pair,round(cmb_prob,rnd))
    end
    
    return huffman_tree
    

HuffmanTreeFirstAttempt (generic function with 1 method)

In [124]:
function huffmanTree( pd; rounding = 5)
    huffman_tree = copy(pd)
    
    while(length(huffman_tree) > 1)
        cmb_prob = 0.0
        pair = Dict()
        
        #get first new item
        m = (k,v) = minvalue(huffman_tree) # get the smallest value form the huffman tree
        push!(pair,k,v) 
        cmb_prob += pop!(huffman_tree,k)
        
        m = (k,v) = minvalue(huffman_tree) # get the smallest value form the huffman tree
        push!(pair,k,v) 
        cmb_prob += pop!(huffman_tree,k)
        
        push!(huffman_tree,pair,round(cmb_prob,rounding))
    end
    
    return huffman_tree
end

huffmanTree (generic function with 1 method)

In [111]:
hufftree_34 = huffmanTree(pd_34)

{{"B"=>0.2,"A"=>0.8}=>1.0}

In [113]:
hufftree2_34 = huffmanTree(pd2_34)

{{"AA"=>0.64,{"AB"=>0.16,{"BB"=>0.04,"BA"=>0.16}=>0.2}=>0.36}=>1.0}

In [119]:
hufftree3_34 = huffmanTree(pd3_34)

{{"AAA"=>0.512,{{{{"BBB"=>0.008,"BAB"=>0.032}=>0.04,{"BBA"=>0.032,"ABB"=>0.032}=>0.064}=>0.104,"ABA"=>0.128}=>0.232,{"BAA"=>0.128,"AAB"=>0.128}=>0.256}=>0.488}=>1.0}

In [160]:
# given a huffman tree create a huffman code
function huffmanCode( huff_tree; rounding = 5)
    huff_code = copy(huff_tree)
    
    # set the first node to the empty string which will let the coding propagate down the tree
    top = collect(keys(huff_code))[1]
    huff_code[top] = ""
        
    while( sum(map((x)->(isa(x,Dict)), collect(keys(huff_code)))) > 0)
        # this is complicated way of asking "Are there any Dictionaries left?", which is
        # equivalent to the question "Are all the letters encoded?"
        for (k,c) in huff_code
            if( isa(k,Dict) )
                kz = collect(keys(k))
                push!(huff_code, kz[1], c*"0")
                push!(huff_code, kz[2], c*"1")
                pop!(huff_code,k) # remove the node, leaving the lower nodes
            end
        end
    end

    return huff_code
end         

huffmanCode (generic function with 1 method)

In [164]:
hc_34 = huffmanCode(hufftree_34)

{"B"=>"0","A"=>"1"}

In [165]:
hc2_34 = huffmanCode(hufftree2_34)

{"AA"=>"0","BB"=>"110","AB"=>"10","BA"=>"111"}

In [166]:
hc3_34 = huffmanCode(hufftree3_34)

{"BAA"=>"110","BBB"=>"10000","AAA"=>"0","AAB"=>"111","BAB"=>"10001","BBA"=>"10010","ABB"=>"10011","ABA"=>"101"}

In [178]:
function averageCodeLength( pd, hc; base = 2 )
    r = log(base, length(pd))
    avg = 0.0
    
    for (k,v) in pd
        avg += v*length(hc[k])
    end
    
    return (avg/r)
end

averageCodeLength (generic function with 1 method)

In [179]:
averageCodeLength( pd_34, hc_34 )

1.0

In [180]:
averageCodeLength( pd2_34, hc2_34 )

0.78

In [181]:
averageCodeLength( pd3_34, hc3_34 )

0.7279999999999999

## 4.16

* $ P = 0.02$
* $ a = 0.90$
* $ \frac{1}{P} = 50 $
* $ n_p = 6 = \log_2(2^{n_p}) \ge \log_2(\frac{1}{P}) \approx 5.6438 $
* $ n = 7 = n_p + 1 $
* $ c = 116 $ from $ c-1 = 115 \le 2^{7}(0.9) = 115.2 \le 116 = c $

And 116 in binary is `1110100`


## LZW encoding

In [199]:
sample = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales lacus nec augue sodales consequat. Nam vel placerat felis, ac posuere orci. Phasellus volutpat feugiat orci. Duis consectetur, orci ut porttitor vehicula, augue lectus egestas nunc, et condimentum est libero vel magna. Aenean tristique turpis vitae rutrum lobortis. Cras cursus ornare velit. Suspendisse lobortis nulla lectus, ac vehicula magna suscipit in. In sit amet dui in velit tristique placerat nec ultricies justo. Donec sed mollis quam. Praesent et sollicitudin eros, sit amet placerat quam. Integer sit amet lobortis turpis, quis viverra urna."

"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales lacus nec augue sodales consequat. Nam vel placerat felis, ac posuere orci. Phasellus volutpat feugiat orci. Duis consectetur, orci ut porttitor vehicula, augue lectus egestas nunc, et condimentum est libero vel magna. Aenean tristique turpis vitae rutrum lobortis. Cras cursus ornare velit. Suspendisse lobortis nulla lectus, ac vehicula magna suscipit in. In sit amet dui in velit tristique placerat nec ultricies justo. Donec sed mollis quam. Praesent et sollicitudin eros, sit amet placerat quam. Integer sit amet lobortis turpis, quis viverra urna."

In [241]:
function createAlphabet( text )
    alphabet = Dict()
    
    for i in 1:length(text) 
        alphabet[text[i:i]] = get(alphabet,text[i:i],0)+1
    end

    return alphabet
end

createAlphabet (generic function with 1 method)

In [242]:
alpha = createAlphabet( sample )

{"A"=>2,"e"=>57,"f"=>2,"m"=>16," "=>91,"u"=>41,"l"=>32,"n"=>25,"r"=>34,"h"=>3,"j"=>1,"d"=>9,"S"=>1,"I"=>2,"g"=>8,"N"=>1,"s"=>47,"D"=>2,"i"=>47,"q"=>7,"P"=>2,"o"=>29,"v"=>10,"c"=>28,"C"=>1,","=>8,"L"=>1,"b"=>4,"p"=>12,"t"=>50,"."=>12,"a"=>41}

In [468]:
# given an alphabet, create an lzw
function lzwEncode( alphabet, text )
    codebook = deepcopy(alphabet)
    code = Array(Any,0)
    
    # initialize the codebook
    lzw_idx = 1
    for (s,c) in codebook
        codebook[s] = lzw_idx
        lzw_idx += 1
    end
    
    # take a snapshot of the codebook which is just the encode alphabet
    ab = deepcopy(codebook)
    
    # first symbol gets encoded and lzw codebook expanded
    push!(code, codebook[text[1:1]])
    push!(codebook,text[1:2], lzw_idx)
    lzw_idx += 1
    
    i = 2
    l = length(text)
    while( i < l  )
    
        run_length = 0
        while( ((i+run_length) < l) && (get(codebook,text[i:i+run_length] ,false) != false ))
                  #( get(codebook,text[i:i+run_length],false) != false ) )
            run_length += 1
        end
        
        # run_length is 1 character past the readable block
        
        # first, push string (or char) to be encoded later
        push!(code, codebook[ text[i:i+run_length-1] ])
        
        # then expand the codebook
        push!(codebook,text[i:i+run_length],lzw_idx)
        lzw_idx += 1
        
        i += (run_length)
    end
    
    # find the fixed length of the code and convert the code to a binary string
    fixed_length = int(ceil(log2(length(codebook))))
    encoded = ""
    for i in 1:length(code)
        encoded = encoded*bin(code[i],fixed_length)
    end
    
    return (ab, fixed_length, encoded)
end

lzwEncode (generic function with 1 method)

In [469]:
lzw_sample = lzwEncode(alpha,sample)

({"A"=>1,"e"=>2,"f"=>3,"m"=>4," "=>5,"u"=>6,"l"=>7,"n"=>8,"r"=>9,"h"=>10,"j"=>11,"d"=>12,"S"=>13,"I"=>14,"g"=>15,"N"=>16,"s"=>17,"D"=>18,"i"=>19,"q"=>20,"P"=>21,"o"=>22,"v"=>23,"c"=>24,"C"=>25,","=>26,"L"=>27,"b"=>28,"p"=>29,"t"=>30,"."=>31,"a"=>32},9,"000011011000010110000001001000000010000000100000000101000010011000011101000010001000000110000100101000001100000010110000000111000100010000000101000010001000010011000011110000000101000100000000000100000000010000011110000011010000000101000011000000010110000001000000010001000000010000011000000011110000110111000000110000001001000110100000001100000100111000010011000010001000011000000010011000001000000001111000000101000000010000000111000110010000011111000000101000000001001010000000010100000000110000110101000110000000010110000001100000100000000000111000000010000010001000000101000000111000100000000011000000000110001011111000001000000111111000110100000000110000001111000000110000000010001011001001011011001011101001011111000111011000111101000000010

In [470]:
length(lzw_sample[3])/lzw_sample[2]

367.0

In [471]:
function lzw_decode( alphabet, fxl, encoded_text )
    ab = copy(alphabet)
    codebook = Dict()
    idx = length(alphabet)+1
    
    #inveret the codebook
    for (c,s) in alphabet
        push!(codebook, bin(s,fxl), c)
    end
#10    
    decoded = codebook[encoded_text[1:fxl]]
    
    # peek 
    new_string = codebook[encoded_text[1:fxl]]
    println(new_string)
    new_string = new_string*codebook[encoded_text[(1+fxl):(fxl+fxl)]]
    push!(codebook,bin(idx,fxl),new_string)
    push!(ab,new_string, idx)
    idx += 1

    for i = 1+fxl:fxl:length(encoded_text)-fxl
        decoded = decoded*codebook[encoded_text[i:i+fxl-1]]
        new_string = codebook[encoded_text[i:(i+fxl-1)]]*codebook[encoded_text[(i+fxl):(i+fxl+fxl-1)]]
        push!(codebook, bin(idx,fxl),new_string)
        push!(ab, new_string, idx)
        idx += 1
    end
    return decoded
end
        

lzw_decode (generic function with 1 method)

In [472]:
lzw_decode(lzw_sample[1], lzw_sample[2], lzw_sample[3])

L


"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Alitquam sodales lacus nec augue sodales consequat. Nam svel placerat felis, ac posuere soorci. Phasellus  volutpat felugiat orcii. Duis consectetur, orcii ut porttitor velhicula, acugue lectus  egestase nunc, et conndipmentus  m est libero vel magna. Aenecan tristitque sotus  rpis cvitae sorutrutm lorbortits. Crats coursus ornare vel litt.  Sus penecdipsse loborrtis cnulla lecctus , acc vel hiculaa lecmagna lecsus cipis ct in. In sit amet conduis in vellitt.  trisstitquue lplaacerat necc vel ultricuies jus to. Donec ased mollits quam. Pratesenect et consolliticuitus  dipn eros, sit  amet plaacceratt quam. Intus  eger sit ameet conlorbortitis ctus  rpis cs,  quis cvivelrrat utr"

In [459]:
length(sample)

626